In [33]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from tqdm import tqdm

# --- Load and Preprocess Data (DO NOT MODIFY AFTER THIS) ---
ndf = pd.read_csv("../Downloads/st_exportd.csv").iloc[:,1:]
for i in range(len(ndf)):
    val = ndf.iloc[i,-3]
    if isinstance(val, str):
        ndf.iloc[i,-3] = val.split("GMT")[0].strip()
    else:
        ndf.iloc[i,-3] = np.nan
ndf['timestamp'] = pd.to_datetime(ndf['timestamp'], errors='coerce')
ndf = ndf.sort_values('timestamp').reset_index(drop=True)

In [40]:
import datetime

In [9]:
# --- Display Results (FULL OUTPUT) ---
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect terminal width
pd.set_option('display.max_colwidth', None)  # Show full content of each column

In [34]:
len(ndf)

127615

In [38]:
ndf[ndf["timestamp"]>"2025-12-14"].head()

,klineacc,spread,spreadper,x,vwap,deviation,ratio,term,sigma,e,h,asset,timestamp,gap,gaplimit
126632,136327040,3.245471e+06,21.387764,0.833578,90278.953125,-0.001986,0.786122,0.558023,0.001524,-0.571918,-0.516766,BTC,2025-12-14 00:03:30,1m,300
126633,60454616,-3.964839e+05,-4.153597,-0.875307,3112.740234,-0.000932,1.041536,1.000672,0.002255,-0.430715,-0.405919,ETH,2025-12-14 00:03:30,1m,300
126634,13732570,-1.427495e+06,-16.985023,-0.753543,2.035120,-0.002516,1.169850,0.449436,0.001342,-0.985701,-0.755523,XRP,2025-12-14 00:03:30,1m,300
126635,18700514,1.523946e+06,27.859571,0.758397,894.148132,0.005158,0.721404,2.158365,0.003004,2.673498,0.990520,BNB,2025-12-14 00:03:30,1m,300
126636,57114864,-9.369018e+06,-398.108002,-0.716057,436.818176,0.012824,4.981080,1.339800,0.017942,4.769938,0.999856,ZEC,2025-12-14 00:03:30,1m,300


In [36]:
ndf.iloc[126630:126635]

,klineacc,spread,spreadper,x,vwap,deviation,ratio,term,sigma,e,h,asset,timestamp,gap,gaplimit
126630,137683680,2.766992e+06,19.700949,0.840489,90276.765625,-0.002075,0.802990,0.596524,0.001563,-0.635936,-0.562126,BTC,2025-12-13 23:59:51,3m,100
126631,13948327,-1.053583e+06,-11.970644,-0.776005,2.034915,-0.002808,1.119706,0.504429,0.001373,-1.155545,-0.819583,XRP,2025-12-13 23:59:51,3m,100
126632,136327040,3.245471e+06,21.387764,0.833578,90278.953125,-0.001986,0.786122,0.558023,0.001524,-0.571918,-0.516766,BTC,2025-12-14 00:03:30,1m,300
126633,60454616,-3.964839e+05,-4.153597,-0.875307,3112.740234,-0.000932,1.041536,1.000672,0.002255,-0.430715,-0.405919,ETH,2025-12-14 00:03:30,1m,300
126634,13732570,-1.427495e+06,-16.985023,-0.753543,2.035120,-0.002516,1.169850,0.449436,0.001342,-0.985701,-0.755523,XRP,2025-12-14 00:03:30,1m,300


# predict h

# predict e

In [46]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

# --- Split Data (DO NOT MODIFY AFTER THIS) ---
train_df = ndf.iloc[:126632, :].copy()
actual_x = ndf.iloc[126632:, :].copy()

# --- Define Gap Groups and Target ---
short_gaps = ['1m', '3m', '5m', '15m', '30m', '1h']
long_gaps = ['2h', '4h', '6h', '8h', '12h']
target = 'e'  # Predict 'e' instead of 'h'
features = ['klineacc', 'spread', 'vwap', 'deviation', 'ratio', 'term', 'sigma']

# --- Helper Functions ---
def get_last_train_timestamp(df, asset, gap):
    """Return the last timestamp for a given asset and gap in train_df."""
    subset = df[(df['asset'] == asset) & (df['gap'] == gap)]
    return subset['timestamp'].max()

def shortlist_actual_x(actual_x, asset, gap, end_duration):
    """Shortlist actual_x entries for evaluation."""
    mask = (
        (actual_x['asset'] == asset) &
        (actual_x['gap'] == gap) &
        (actual_x['timestamp'] <= end_duration)
    )
    return actual_x[mask]

# --- Training ---
models = {}
unique_assets = set()
for key in train_df['asset'].unique():
    unique_assets.add(key)

for asset in tqdm(unique_assets, desc="Training assets"):
    for gap in short_gaps + long_gaps:
        # Filter train_df for this asset and gap
        train_subset = train_df[(train_df['asset'] == asset) & (train_df['gap'] == gap)]
        if len(train_subset) == 0:
            continue

        # Prepare data
        X_train = train_subset[features]
        y_train = train_subset[target]

        # Train XGBoost model
        model = XGBRegressor(
            objective='reg:squarederror',
            n_estimators=200,
            max_depth=6,
            learning_rate=0.05,
            reg_lambda=1,  # L2 regularization to prevent overfitting
            random_state=42
        )
        model.fit(X_train, y_train)

        # Save model
        model_key = f"{asset}_{gap}"
        models[model_key] = model

# --- Save All Models to Disk ---
with open('trained_models_e.pkl', 'wb') as f:
    pickle.dump(models, f)

print("Training complete. Models saved to 'trained_models_e.pkl'.")

Training assets: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:45<00:00,  1.63it/s]


Training complete. Models saved to 'trained_models_e.pkl'.


In [48]:
# --- Evaluation ---
results = []
predicted_results = []  # Store predicted vs actual 'e' and 'h'

# Load models
with open('trained_models_e.pkl', 'rb') as f:
    models = pickle.load(f)

print(unique_assets,len(unique_assets))

for asset in tqdm(unique_assets, desc="Evaluating assets"):
    for gap in short_gaps + long_gaps:
        model_key = f"{asset}_{gap}"
        if model_key not in models:
            continue

        # Get last train timestamp
        last_train_ts = get_last_train_timestamp(train_df, asset, gap)
        if pd.isna(last_train_ts):
            continue

        # Calculate end_duration
        if gap in short_gaps:
            duration_minutes = 300
        else:
            duration_minutes = 7200
        end_duration = last_train_ts + pd.Timedelta(minutes=duration_minutes)

        # Shortlist actual_x
        eval_subset = shortlist_actual_x(actual_x, asset, gap, end_duration)
        if len(eval_subset) == 0:
            continue

        # Prepare data
        X_eval = eval_subset[features]
        y_eval_e = eval_subset[target]  # Actual 'e'
        y_eval_h = eval_subset['h']    # Actual 'h' (for directional accuracy)

        # Predict 'e'
        model = models[model_key]
        y_pred_e = model.predict(X_eval)
        y_pred_h = np.tanh(y_pred_e)    # Compute 'h' from predicted 'e'

        # Calculate metrics
        mae_e = mean_absolute_error(y_eval_e, y_pred_e)
        mae_h = mean_absolute_error(y_eval_h, y_pred_h)
        directional_acc = np.mean((np.sign(y_eval_h) == np.sign(y_pred_h)).astype(float))

        # Store evaluation results
        results.append({
            'asset': asset,
            'gap': gap,
            'last_train_timestamp': last_train_ts,
            'end_duration': end_duration,
            'n_eval_samples': len(eval_subset),
            'MAE_e': mae_e,
            'MAE_h': mae_h,
            'Directional_Accuracy': directional_acc
        })

        # Store predicted vs actual 'e' and 'h'
        eval_subset = eval_subset.reset_index(drop=True)
        for idx in range(len(eval_subset)):
            predicted_results.append({
                'asset': asset,
                'gap': gap,
                'timestamp': eval_subset.loc[idx, 'timestamp'],
                'actual_e': eval_subset.loc[idx, target],
                'predicted_e': y_pred_e[idx],
                'actual_h': eval_subset.loc[idx, 'h'],
                'predicted_h': y_pred_h[idx],
                'absolute_error_e': abs(eval_subset.loc[idx, target] - y_pred_e[idx]),
                'absolute_error_h': abs(eval_subset.loc[idx, 'h'] - y_pred_h[idx]),
                'last_train_timestamp': last_train_ts
            })

# --- Save Results ---
results_df = pd.DataFrame(results)
predicted_df = pd.DataFrame(predicted_results)

# Find the entry with the biggest error for 'e' and 'h'
biggest_error_e_df = predicted_df.loc[predicted_df.groupby(['asset', 'gap'])['absolute_error_e'].idxmax()]
biggest_error_h_df = predicted_df.loc[predicted_df.groupby(['asset', 'gap'])['absolute_error_h'].idxmax()]

# Save to CSV
results_df.to_csv('evaluation_results_e.csv', index=False)
predicted_df.to_csv('predicted_vs_actual_e_h.csv', index=False)
biggest_error_e_df.to_csv('biggest_error_e_entries.csv', index=False)
biggest_error_h_df.to_csv('biggest_error_h_entries.csv', index=False)

print("Evaluation complete. Results saved to:")
print("- evaluation_results_e.csv (summary)")
print("- predicted_vs_actual_e_h.csv (all predicted vs actual 'e' and 'h' values)")
print("- biggest_error_e_entries.csv (entries with the biggest error for 'e')")
print("- biggest_error_h_entries.csv (entries with the biggest error for 'h')")

{'ASR', 'APT', 'RENDER', 'FIL', 'ENSO', 'TWT', 'ZRO', 'ASTER', 'ORCA', 'MORPHO', 'AXS', 'LTC', 'BNSOL', 'BANANA', 'LPT', 'MOVR', '0G', 'PAXG', 'BNB', 'KSM', 'LINK', 'VIRTUAL', 'MLN', 'BTC', 'BCH', 'PENDLE', 'EUL', 'AUCTION', 'XNO', 'ENS', 'VANA', 'NMR', 'ZEN', 'ORDI', 'INJ', 'QNT', 'METIS', 'SSV', 'FARM', 'PYR', 'COMP', 'CAKE', 'ILV', 'DOT', 'TON', 'ATOM', 'UNI', 'SOL', 'WBTC', 'TAO', 'DCR', 'ZEC', 'ETH', 'DASH', 'TRUMP', 'WBETH', 'TRB', 'RAY', 'ETC', 'SUI', 'NEAR', 'ALCX', 'EGLD', 'SANTOS', 'AVAX', 'MMT', 'AR', 'AAVE', 'OG', 'XRP', 'NEO', 'ICP', 'BERA', 'GIGGLE'} 74


Evaluating assets: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:10<00:00,  7.05it/s]

Evaluation complete. Results saved to:
- evaluation_results_e.csv (summary)
- predicted_vs_actual_e_h.csv (all predicted vs actual 'e' and 'h' values)
- biggest_error_e_entries.csv (entries with the biggest error for 'e')
- biggest_error_h_entries.csv (entries with the biggest error for 'h')


In [59]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

# --- Load and Preprocess Data (DO NOT MODIFY) ---
# (Assume ndf, train_df, actual_x are already loaded and preprocessed)

# --- Load Trained Models ---
with open('trained_models_e.pkl', 'rb') as f:
    models = pickle.load(f)

# --- Define Gap Groups and Target ---
short_gaps = ['1m', '3m', '5m', '15m', '30m', '1h']
long_gaps = ['2h', '4h', '6h', '8h', '12h']
target = 'e'  # Predict 'e' and compute 'h'
features = ['klineacc', 'spread', 'vwap', 'deviation', 'ratio', 'term', 'sigma']

# --- Helper Functions ---
def get_last_train_timestamp(df, asset, gap):
    """Return the last timestamp for a given asset and gap in train_df."""
    subset = df[(df['asset'] == asset) & (df['gap'] == gap)]
    return subset['timestamp'].max()

def shortlist_actual_x(actual_x, asset, gap, end_duration):
    """Shortlist actual_x entries for evaluation."""
    mask = (
        (actual_x['asset'] == asset) &
        (actual_x['gap'] == gap) &
        (actual_x['timestamp'] <= end_duration)
    )
    return actual_x[mask]

# --- Evaluation ---
results = []
predicted_results = []  # Store ALL predicted vs actual 'e' and 'h' for EVERY ROW

# Extract unique assets from model keys
unique_assets = set()
for key in models.keys():
    asset = key.split('_')[0]
    unique_assets.add(asset)

for asset in tqdm(unique_assets, desc="Evaluating assets"):
    for gap in short_gaps + long_gaps:
        model_key = f"{asset}_{gap}"
        if model_key not in models:
            continue

        # Get last train timestamp
        last_train_ts = get_last_train_timestamp(train_df, asset, gap)
        if pd.isna(last_train_ts):
            continue

        # Calculate end_duration
        if gap in short_gaps:
            duration_minutes = 300
        else:
            duration_minutes = 7200
        end_duration = last_train_ts + pd.Timedelta(minutes=duration_minutes)

        # Shortlist actual_x
        eval_subset = shortlist_actual_x(actual_x, asset, gap, end_duration)
        if len(eval_subset) == 0:
            continue

        # Prepare data
        X_eval = eval_subset[features]
        y_eval_e = eval_subset[target]  # Actual 'e'
        y_eval_h = eval_subset['h']    # Actual 'h'

        # Predict 'e' and compute 'h'
        model = models[model_key]
        y_pred_e = model.predict(X_eval)
        y_pred_h = np.tanh(y_pred_e)  # Compute 'h' from predicted 'e'

        # Calculate metrics (for summary)
        mae_e = mean_absolute_error(y_eval_e, y_pred_e)
        mae_h = mean_absolute_error(y_eval_h, y_pred_h)
        directional_acc = np.mean((np.sign(y_eval_h) == np.sign(y_pred_h)).astype(float))

        # Store evaluation results (summary)
        results.append({
            'asset': asset,
            'gap': gap,
            'last_train_timestamp': last_train_ts,
            'end_duration': end_duration,
            'n_eval_samples': len(eval_subset),
            'MAE_e': mae_e,
            'MAE_h': mae_h,
            'Directional_Accuracy': directional_acc
        })

        # Store predicted vs actual 'e' and 'h' for EVERY ROW in eval_subset
        eval_subset = eval_subset.reset_index(drop=True)
        for idx in range(len(eval_subset)):
            predicted_results.append({
                'asset': asset,
                'gap': gap,
                'timestamp': eval_subset.loc[idx, 'timestamp'],
                'actual_e': eval_subset.loc[idx, target],
                'predicted_e': y_pred_e[idx],
                'actual_h': eval_subset.loc[idx, 'h'],
                'predicted_h': y_pred_h[idx],
                'absolute_error_e': abs(eval_subset.loc[idx, target] - y_pred_e[idx]),
                'absolute_error_h': abs(eval_subset.loc[idx, 'h'] - y_pred_h[idx]),
                'last_train_timestamp': last_train_ts
            })

# --- Save Results ---
results_df = pd.DataFrame(results)
predicted_df = pd.DataFrame(predicted_results)

# Save to CSV
results_df.to_csv('evaluation_results_e.csv', index=False)
predicted_df.to_csv('predicted_vs_actual_e_h_detailed.csv', index=False)  # ALL ROWS

print("Evaluation complete. Results saved to:")
print("- evaluation_results_e.csv (summary metrics)")
print("- predicted_vs_actual_e_h_detailed.csv (EVERY predicted vs actual 'e' and 'h' for all rows)")

Evaluating assets: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:10<00:00,  6.93it/s]


Evaluation complete. Results saved to:
- evaluation_results_e.csv (summary metrics)
- predicted_vs_actual_e_h_detailed.csv (EVERY predicted vs actual 'e' and 'h' for all rows)


In [60]:
f=['BTC','ETH','SOL','ZEC','BNB', 'XRP','PAXG','BCH','PENDLE','AAVE','TAO']

In [61]:
results_df.sort_values(['asset', 'gap'])[results_df["asset"].isin(f)]

/var/folders/5z/34btjl757md3wk7szryjd3ym0000gn/T/ipykernel_63331/944919742.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results_df.sort_values(['asset', 'gap'])[results_df["asset"].isin(f)]


,asset,gap,last_train_timestamp,end_duration,n_eval_samples,MAE_e,MAE_h,Directional_Accuracy
94,AAVE,2h,2025-12-13 18:41:13,2025-12-18 18:41:13,3,0.014574,0.013360,1.000000
95,AAVE,4h,2025-12-13 17:36:34,2025-12-18 17:36:34,1,1.629389,0.624014,1.000000
30,BCH,2h,2025-12-13 18:41:13,2025-12-18 18:41:13,3,0.054531,0.054281,1.000000
31,BCH,4h,2025-12-13 17:36:34,2025-12-18 17:36:34,1,0.054606,0.054544,1.000000
15,BNB,15m,2025-12-13 23:39:30,2025-12-14 04:39:30,2,0.271477,0.219930,1.000000
17,BNB,1h,2025-12-13 23:55:40,2025-12-14 04:55:40,1,0.139224,0.003071,1.000000
12,BNB,1m,2025-12-13 23:50:36,2025-12-14 04:50:36,21,0.108729,0.040751,1.000000
18,BNB,2h,2025-12-13 18:41:13,2025-12-18 18:41:13,3,0.022479,0.021701,1.000000
16,BNB,30m,2025-12-13 23:55:37,2025-12-14 04:55:37,1,0.058323,0.053554,1.000000
13,BNB,3m,2025-12-13 23:59:51,2025-12-14 04:59:51,12,0.166952,0.080795,1.000000


In [62]:
predicted_df.sort_values(['asset', 'gap'])[predicted_df["asset"].isin(f)]

/var/folders/5z/34btjl757md3wk7szryjd3ym0000gn/T/ipykernel_63331/1472433255.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  predicted_df.sort_values(['asset', 'gap'])[predicted_df["asset"].isin(f)]


,asset,gap,timestamp,actual_e,predicted_e,actual_h,predicted_h,absolute_error_e,absolute_error_h,last_train_timestamp
398,AAVE,2h,2025-12-14 00:45:02,-0.011704,-0.040797,-0.011703,-0.040775,0.029094,0.029072,2025-12-13 18:41:13
399,AAVE,2h,2025-12-14 06:48:51,-0.088756,-0.092669,-0.088523,-0.092405,0.003913,0.003881,2025-12-13 18:41:13
400,AAVE,2h,2025-12-14 12:52:48,-0.655190,-0.665906,-0.575154,-0.582280,0.010716,0.007127,2025-12-13 18:41:13
401,AAVE,4h,2025-12-14 05:40:38,-0.352648,-1.982037,-0.338722,-0.962736,1.629389,0.624014,2025-12-13 17:36:34
142,BCH,2h,2025-12-14 00:45:02,0.038621,0.074506,0.038602,0.074368,0.035885,0.035767,2025-12-13 18:41:13
143,BCH,2h,2025-12-14 06:48:51,0.023917,0.087847,0.023912,0.087621,0.063930,0.063709,2025-12-13 18:41:13
144,BCH,2h,2025-12-14 12:52:48,0.046402,0.110179,0.046368,0.109735,0.063777,0.063367,2025-12-13 18:41:13
145,BCH,4h,2025-12-14 05:40:38,-0.002657,-0.057263,-0.002657,-0.057200,0.054606,0.054544,2025-12-13 17:36:34
71,BNB,15m,2025-12-14 01:58:34,-0.462918,-0.732353,-0.432459,-0.624503,0.269435,0.192043,2025-12-13 23:39:30
72,BNB,15m,2025-12-14 04:17:32,-0.172468,-0.445988,-0.170778,-0.418595,0.273520,0.247817,2025-12-13 23:39:30


In [64]:
biggest_error_e_df.sort_values(['asset', 'gap'])[biggest_error_e_df["asset"].isin(f)]

,asset,gap,timestamp,actual_e,predicted_e,actual_h,predicted_h,absolute_error_e,absolute_error_h,last_train_timestamp
398,AAVE,2h,2025-12-14 00:45:02,-0.011704,-0.040797,-0.011703,-0.040775,0.029094,0.029072,2025-12-13 18:41:13
401,AAVE,4h,2025-12-14 05:40:38,-0.352648,-1.982037,-0.338722,-0.962736,1.629389,0.624014,2025-12-13 17:36:34
143,BCH,2h,2025-12-14 06:48:51,0.023917,0.087847,0.023912,0.087621,0.063930,0.063709,2025-12-13 18:41:13
145,BCH,4h,2025-12-14 05:40:38,-0.002657,-0.057263,-0.002657,-0.057200,0.054606,0.054544,2025-12-13 17:36:34
72,BNB,15m,2025-12-14 04:17:32,-0.172468,-0.445988,-0.170778,-0.418595,0.273520,0.247817,2025-12-13 23:39:30
74,BNB,1h,2025-12-14 02:59:38,-2.526582,-2.665806,-0.987303,-0.990374,0.139224,0.003071,2025-12-13 23:55:40
24,BNB,1m,2025-12-14 00:03:30,2.673498,3.449329,0.990520,0.997984,0.775830,0.007463,2025-12-13 23:50:36
75,BNB,2h,2025-12-14 00:45:02,0.214212,0.251287,0.210994,0.246128,0.037075,0.035134,2025-12-13 18:41:13
73,BNB,30m,2025-12-14 02:59:33,-0.264596,-0.322919,-0.258590,-0.312144,0.058323,0.053554,2025-12-13 23:55:37
46,BNB,3m,2025-12-14 00:43:37,2.857833,2.163942,0.993434,0.973953,0.693891,0.019481,2025-12-13 23:59:51


In [65]:
biggest_error_h_df.sort_values(['absolute_error_h'])[biggest_error_h_df["asset"].isin(f)]

/var/folders/5z/34btjl757md3wk7szryjd3ym0000gn/T/ipykernel_63331/3434181425.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  biggest_error_h_df.sort_values(['absolute_error_h'])[biggest_error_h_df["asset"].isin(f)]


,asset,gap,timestamp,actual_e,predicted_e,actual_h,predicted_h,absolute_error_e,absolute_error_h,last_train_timestamp
78,BNB,4h,2025-12-14 05:40:38,0.069589,0.069980,0.069477,0.069866,0.000390,0.000389,2025-12-13 17:36:34
74,BNB,1h,2025-12-14 02:59:38,-2.526582,-2.665806,-0.987303,-0.990374,0.139224,0.003071,2025-12-13 23:55:40
243,SOL,1h,2025-12-14 02:59:38,-3.132092,-5.822245,-0.996201,-0.999982,2.690153,0.003782,2025-12-13 23:55:40
452,XRP,1h,2025-12-14 02:59:38,-2.482893,-2.242247,-0.986152,-0.977687,0.240646,0.008465,2025-12-13 23:55:40
137,BTC,1h,2025-12-14 02:59:38,0.600509,0.612831,0.537412,0.546117,0.012322,0.008705,2025-12-13 23:55:40
241,SOL,15m,2025-12-14 04:17:32,0.113389,0.124494,0.112905,0.123855,0.011105,0.010950,2025-12-13 23:39:30
368,ETH,30m,2025-12-14 02:59:33,-0.240155,-0.253865,-0.235642,-0.248549,0.013711,0.012907,2025-12-13 23:55:37
366,ETH,15m,2025-12-14 01:58:34,-0.722438,-0.698141,-0.618417,-0.603186,0.024297,0.015231,2025-12-13 23:39:30
136,BTC,30m,2025-12-14 02:59:33,0.129989,0.102874,0.129262,0.102512,0.027115,0.026749,2025-12-13 23:55:37
398,AAVE,2h,2025-12-14 00:45:02,-0.011704,-0.040797,-0.011703,-0.040775,0.029094,0.029072,2025-12-13 18:41:13
